# ClickHouse   

## Идем в песочницу 
https://fiddle.clickhouse.com

## Создаем таблицу `employee_data`


```sql
CREATE TABLE employee_data (
    Age Int32,
    Salary Int32,
    Experience Int32,
    City String,
    Gender String,
    Profession String,
    Industry String,
    Satisfaction_Level Float32,
    Project_Count Int32,
    Bonuses Float32,
    Start_Date Date
) ENGINE = MergeTree()
ORDER BY Age;
```

## Заполняем таблицу данными 

```sql
INSERT INTO employee_data (
    Age, 
    Salary, 
    Experience, 
    City, 
    Gender, 
    Profession, 
    Industry, 
    Satisfaction_Level, 
    Project_Count, 
    Bonuses, 
    Start_Date
) 
VALUES
(35, 70000, 10, 'Moscow', 'M', 'Engineer', 'Energy', 8.2, 12, 120000, '2020-01-11'),
(45, NULL, 15, 'SPb', 'F', 'Manager', 'Finance', 6.8, NULL, 150000, '2020-02-03'),
(29, 500000, 7, 'Novosibirsk', 'M', 'Developer', 'Technology', 9.0, 22, NULL, '2020-03-21'),
(NULL, 80000, NULL, 'Yekaterinburg', 'F', 'Analyst', 'Technology', 7.5, 14, 130000, '2020-04-05'),
(40, 850000, 14, 'Kazan', 'M', 'Engineer', 'Natural Resources', NULL, 20, 675000, '2020-05-04'),
(33, NULL, 11, 'Omsk', 'F', 'Senior Specialist', 'Natural Resources', 7.0, 17, 125000, NULL),
(28, 60000, NULL, 'Samara', 'M', 'Technician', 'Energy', 8.9, 23, NULL, '2020-07-01'),
(36, 780000, 13, 'Krasnoyarsk', 'F', 'Manager', 'Technology', 6.5, NULL, 130000, '2020-08-01'),
(NULL, 92000, 16, 'Volgograd', 'M', 'Analyst', 'AI', NULL, 15, 140000, '2020-09-06'),
(31, 670000, 9, NULL, 'F', 'Developer', 'Finance', 8.3, 21, 120000, '2019-10-07'),
(35, NULL, NULL, 'Moscow', 'M', 'Engineer', 'Energy', 8.2, NULL, 120000, NULL),
(19, 67787, 9, 'Saratov', 'M', 'Manager', 'Finance', NULL, 21, 120150, '2021-16-01'),
(30, 95000, 14, 'Kazan', 'M', 'Engineer', 'Natural Resources', 8.1, 20, 145000, '2020-05-01'),
(34, 40000, 6, 'Moscow', 'F', 'Analyst', 'Finance', 8.2, 12, 120000, '2020-01-08'),
(15, NULL, 15, 'SPb', 'F', 'Manager', 'Finance', 6.8, NULL, 150000, '2018-02-01'),
(79, 520000, 7, 'Novosibirsk', 'F', 'Developer', 'Finance', 9.0, 22, NULL, '2016-03-21'),
(NULL,60000, NULL, 'Yekaterinburg', 'F', 'Analyst', 'AI', 7.5, 14, 130000, '2020-04-23'),
(20, 351000, 14, 'Kazan', 'F', 'Engineer', 'Natural Resources', NULL, 20, 675000, '2019-05-06'),
(53, NULL, 11, 'Omsk', 'F', 'Senior Specialist', 'Natural Resources', 7.0, 17, 125000, NULL),
(48, 20000, NULL, 'Samara', 'M', 'Technician', 'AI', 8.9, 23, NULL, '2021-07-08'),
(36, 80000, 13, 'Krasnoyarsk', 'M', 'Manager', 'Technology', 6.5, NULL, 130000, '2019-08-11'),
(NULL, 12000, 16, 'Volgograd', 'M', 'Analyst', 'AI', NULL, 15, 140000, '2017-09-19'),
(21, 67000, 9, NULL, 'F', 'Developer', 'Finance', 8.3, 21, 120000, '2021-10-16'),
(65, NULL, NULL, 'Moscow', 'M', 'Engineer', 'Energy', 8.2, NULL, 120000, NULL),
(39, 60700, 9, 'Saratov', 'M', 'Manager', 'Finance', NULL, 21, 120150, '2022-10-13'),
(36, 90000, 14, 'Kazan', 'M', 'Analyst', 'AI', 8.1, 20, 145000, '2010-05-11');

```

## Проверяем, что все корректно 

```sql
select *
from employee_data
```

## Работаем с датой


1.Извлечение года, месяца и дня из даты.

```sql
SELECT toYear(Start_Date), 
       toMonth(Start_Date),
       toDayOfYear(Start_Date),
       toDayOfMonth(Start_Date),
       toDayOfWeek(Start_Date)
FROM employee_data;
```

2.Форматируем даты в определённый строковый формат.

```sql
SELECT formatDateTime(Start_Date, '%W-%D-%M-%Y')
FROM employee_data
```

 3.Вычисляем разницу в днях между `Start_Date` и текущей датой.

```sql
SELECT dateDiff(day, Start_Date, now())
FROM employee_data;
```

4.Получить начало месяца и года для каждой даты, используйте функции `toStartOfMonth()` и `toStartOfYear()`

```sql
SELECT toStartOfMonth(Start_Date),
       toStartOfYear(Start_Date)
FROM employee_data;
```

## Работаем с комбинаторными функциями. 

1.Найдем среднюю зарплату (Salary) для сотрудников с опытом работы больше 10 лет в каждой отрасли.
```sql 
SELECT 
    Industry,
    avgIf(Salary, Experience > 10) AS AvgSalary
FROM employee_data
GROUP BY Industry;
```
2.Посчитаем количество сотрудников в каждой отрасли с уровнем удовлетворённости ниже 6.0.
```sql 
SELECT 
    Industry,
    countIf(Satisfaction_Level > 6) AS Count
FROM employee_data
GROUP BY Industry;
```
3.Получим средний бонус (Bonuses) для сотрудников в каждой отрасли, где уровень удовлетворённости больше среднего уровня удовлетворённости по всем отраслям.
```sql
SELECT 
    Industry,
    round(avgIf(Bonuses, Satisfaction_Level > (SELECT avg(Satisfaction_Level) FROM employee_data))) AS AvgBonuses
FROM employee_data
GROUP BY Industry;

```
4.Найдем максимальный и минимальный возраст сотрудников для каждой профессии в каждой отрасли, учитывая только сотрудников, у которых опыт больше 5 лет и зарплата не менее 50000.
```sql
SELECT 
    Industry,
    Profession,
    minIf(Age, Experience > 5 AND Salary > 50000) AS MinAge,
    maxIf(Age, Experience > 5 AND Salary > 50000) AS MaxAge
FROM employee_data
GROUP BY Industry, Profession;
```

## Работаем с массивами 

1.Для каждой отрасли найдем список уникальных профессий, которыми заняты сотрудники этой отрасли, без учёта пола.
```sql
SELECT 
    Industry,
    groupUniqArray(Profession) AS unique_professions
FROM employee_data
GROUP BY Industry
ORDER BY Industry;
```        
2.Составим список уникальных профессий, которыми заняты только женщины в каждой отрасли.
```sql
SELECT 
    Industry,
    groupUniqArrayIf(Profession, Gender = 'F') AS unique_female_professions
FROM employee_data
GROUP BY Industry
ORDER BY Industry;  
```
3.Определим количество уникальных профессий среди женщин в каждой отрасли, то есть посчитаем уникальные профессии из списка профессий, где работают женщины (п2).
```sql
SELECT 
    Industry,
    length(groupUniqArrayIf(Profession, Gender = 'F')) AS unique_female_professions_count
FROM employee_data
GROUP BY Industry
ORDER BY Industry;

ИЛИ ТАК:

SELECT 
    Industry,
    uniqExactIf(Profession, Gender = 'F') AS unique_female_professions_count
FROM employee_data
GROUP BY Industry
ORDER BY Industry;
```

## Сортировка и фильтрация

1.Соберем массив зарплат для каждой отрасли, включив в него только зарплаты выше 100,000. Отфильтрованный массив отсортируйте в порядке возрастания, чтобы зарплаты располагались от меньших к большим.
```sql
SELECT 
    Industry,
    arraySort(arrayFilter(Salary -> Salary > 100000,
                          groupArray(Salary))
             ) AS filtered_sorted_salaries
FROM employee_data
GROUP BY Industry
ORDER BY Industry
```
2.Создадим альтернативный массив зарплат для каждой отрасли, также включающий только значения выше 100,000, но используя фильтрацию в самой функции и отсортируем массив по возрастанию.
```sql
SELECT 
    Industry,
    arraySort(groupArrayIf(Salary, Salary > 100000)) AS f_s_s
FROM employee_data
GROUP BY Industry
ORDER BY Industry
```

3.Соберем массив зарплат сотрудников в каждой отрасли, отсортировав его по убыванию. 
```sql 
SELECT 
 Industry,
 arrayReverseSort(groupArray(Salary)) AS sorted_salaries
FROM employee_data
GROUP BY Industry
ORDER BY Industry
```

4.Извлечем топ-3 зарплаты для каждой отрасли.
```sql
SELECT Industry,
       arraySlice(arr_sort, -3, 3) AS top_salary
FROM   (SELECT 
          Industry,
          arraySort(groupArray(Salary)) AS arr_sort
       FROM employee_data
       GROUP BY Industry
       ORDER BY Industry)
```



## Используем lambda функции 

1.Составим массив зарплат для каждой отрасли, включив только те зарплаты, которые превышают 50,000. Для анализа возможного повышения зарплат увеличим каждое значение на 10% и округлим результат до одного знака после запятой.
```sql
SELECT Industry,
       arrayMap(x -> round(x * 1.1, 1), arrayFilter(x -> x > 50000, salaries)) AS adjusted_salaries
FROM  (SELECT
          Industry,
          groupArray(Salary) AS salaries
       FROM employee_data
       GROUP BY Industry)
````       
2.Составим массив опыта сотрудников для каждой отрасли, включив только сотрудников с опытом работы более 5 лет. В результате для каждого сотрудника добавим к их текущему опыту 2 года.
```sql
SELECT Industry,
       arrayMap((x -> x + 2), arrayFilter(x -> x > 5, exps)) AS adjusted_experience
FROM  (SELECT
          Industry,
          groupArray(Experience) AS exps
       FROM employee_data
       GROUP BY Industry)
```

## Работаем с CTE 

1. **CTE**:
- Сгруппируем данные по `City` и `Gender`.
- Рассчитаем среднюю зарплату (`AVG(Salary)`) и общее количество сотрудников (`COUNT(*)`) для каждой комбинации города и пола.

2. **Основной запрос**:
- Используем подзапрос, чтобы объединить его результаты с таблицей `employee_data`, сопоставив сотрудников с их городом и полом.
- Отфильтруем данные так, чтобы вывести только тех сотрудников, у которых зарплата выше средней зарплаты по их городу и полу.



```sql
with city_gender_stats as (SELECT City,
             Gender,
             AVG(Salary) AS avg_salary,
             COUNT(*) AS employee_count
FROM employee_data
GROUP BY City, Gender
order by City) 

SELECT e.City,
       e.Gender,
       e.Age,
       e.Salary,
       stats.avg_salary
FROM employee_data AS e
JOIN city_gender_stats AS stats
ON e.City = stats.City AND e.Gender = stats.Gender
WHERE e.Salary > stats.avg_salary;
```

## Оконные функции 

1.1. Для каждой профессии пронумеруем сотрудников, начиная с самого высокого уровня зарплаты, используя функцию `ROW_NUMBER()`.
```sql
SELECT *,
       row_number() over (partition by Profession order by Salary desc) as rnk
FROM employee_data 
order by Profession, Salary desc
```
2.Используем функцию `RANK()`, создадим ранжирование сотрудников в каждой отрасли на основе их зарплаты (начиная с самой высокой).
```sql
SELECT Industry, Profession, Salary,
       rank() over (partition by Industry order by Salary desc) as rnk
FROM employee_data 
order by Industry, Salary desc
```
3.Рассчитаем скользящее среднее уровня удовлетворённости для текущей строки и двух предыдущих сотрудников с такой же профессией, упорядоченных по зарплате
```sql
SELECT Profession, Salary, Satisfaction_Level,
       avg(Satisfaction_Level) over (partition by Profession order by Salary rows between 2 preceding and current row) as avg_sl
FROM employee_data 
order by Profession, Salary desc
```
4.Рассчитаем cумму зарплат всех сотрудников в рамках города.
```sql
SELECT 
    *,
    sum(Salary) OVER (PARTITION BY City) AS city_salary
FROM employee_data
WHERE Salary IS NOT NULL
ORDER BY Profession, city_salary DESC;
```

## limit by

1.Отобразим только одну запись для каждой уникальной комбинации профессии (Profession) и города (City).
```sql
SELECT DISTINCT(Profession, City)
FROM employee_data
order by City
```
2.Отсортируем данные так, чтобы для каждой профессии результаты отображались в алфавитном порядке профессий, а в рамках каждой профессии записи городов сортировались по убыванию.
```sql
SELECT Profession, City
FROM employee_data
order by Profession, City desc
```
3.Используем оператор LIMIT BY для ограничения количества записей до одной на каждую комбинацию профессии и города.
```sql
SELECT Profession, City
FROM employee_data
order by Profession, City desc
limit 1 by Profession, City
```

## any при группировке

Сгруппируем записи по профессии и городу и используйте агрегатную функцию `any` для выбора произвольного значения из каждой группы для следующих столбцов: `Salary`, `Experience`, `Satisfaction_Level`

```sql
SELECT
    Profession,
    City,
    any(Salary) AS salary_sample,
    any(Experience) AS experience_sample,
    any(Satisfaction_Level) AS satisfaction_sample
FROM employee_data
GROUP BY Profession, City
order by Profession, City
```

## Работа со строками

1.Создадим сокращённое обозначение для каждой записи, используя инициалы города (`City`) и профессии (`Profession`). Инициалы должны быть в формате "C.P.", где первая буква города и первая буква профессии указаны в верхнем регистре, разделённые точками. Например, для города "Moscow" и профессии "Engineer" должно получиться "M.E.".
```sql
SELECT
    Profession,
    City,
    CONCAT(UPPER(SUBSTRING(City, 1, 1)), '.', UPPER(SUBSTRING(Profession, 1, 1)), '.') AS abbreviation,
    any(Salary) AS salary_sample,
    any(Experience) AS experience_sample,
    any(Satisfaction_Level) AS satisfaction_sample
FROM
    employee_data
GROUP BY
    Profession,
    City;
```    
2.Отфильтруем записи по названию города. Выберим только те записи, где город начинается с буквы "S" или "K". Используем регулярное выражение для поиска городов с нужной начальной буквой.
```sql
Select City, Age, Profession
from employee_data
where City REGEXP '^(S|K)'
```


## arrayJoin 


1. **Внутри CTE**:
   - Сгруппируем данные по полю `Gender`, чтобы агрегировать информацию по полу сотрудников.
   - Посчитаем общую сумму зарплат (`Salary`) для каждого пола и назовем этот столбец `sum_salary`.
   - Создадим массив, содержащий уникальные профессии (`Profession`) для каждого пола, используя функцию `groupUniqArray`.
2. **Во внешнем запросе**:
   - Используем функцию `arrayJoin` для развертывания массива `arr_profession`, чтобы каждая профессия отображалась как отдельная строка.


```sql
WITH cte AS (
SELECT Gender,
       SUM(Salary) AS sum_salary,
       groupUniqArray(Profession) AS arr_profession
FROM employee_data
GROUP BY Gender)

SELECT Gender,
       sum_salary,
       arrayJoin(arr_profession) AS profession
FROM cte
```

## Удаление таблицы

1.Удалим всех сотрудников из Москвы

```sql
ALTER TABLE employee_data DELETE WHERE City = 'Moscow'
OPTIMIZE TABLE employee_data FINAL
```

2.Очистим таблицу `employee_data`, удалив все записи, но оставив саму таблицу

```sql
TRUNCATE TABLE employee_data;
```

3.Полностью удалим таблицу `employee_data` вместе со всей её структурой и данными

```sql 
DROP TABLE employee_data;
```

## engine 

Исследуем различия в обработке данных между таблицами с движками `MergeTree` и `ReplacingMergeTree`, используя их поведение при наличии дубликатов. Выполним следующие шаги:

1. **Создать две таблицы** с одинаковой структурой, но разными типами движков:
- Таблицу `employee_data_mt` на базе движка `MergeTree`.
- Таблицу `employee_data_rmt` на базе движка `ReplacingMergeTree`, указав в качестве ключа `Start_Date` для замены дубликатов.
2. **Заполнить таблицы данными**:
- Вставить базовые записи для каждой таблицы, указав значения для `Gender`, `City` и `Start_Date`.
- Добавить дополнительные записи в `employee_data_rmt`, которые имеют дубликаты по `Start_Date`, чтобы протестировать поведение `ReplacingMergeTree`.
3. **Проанализировать различия в хранении данных**:
- Выполнить запрос, который подсчитает количество записей в каждой таблице
- Проанализировать результат и ответить на вопросы:
   - Почему количество записей в `employee_data_mt` и `employee_data_rmt` отличается?
   - Какая таблица хранит уникальные записи по ключу `Start_Date`, а какая сохраняет все вставленные данные, включая дубликаты?

```sql
 CREATE TABLE employee_data_mt (
   Gender String,
   City String,
   Start_Date DateTime
 )
 ENGINE = MergeTree()
 ORDER BY (Gender, City);

 CREATE TABLE employee_data_rmt (
   Gender String,
   City String,
   Start_Date DateTime
 )
 ENGINE = ReplacingMergeTree(Start_Date)
 ORDER BY (Gender, City);

-- Вставка данных в таблицу employee_data_mt
INSERT INTO employee_data_mt (Gender, City, Start_Date) VALUES 
    ('M', 'Moscow', '2023-01-01 08:00:00'),
    ('F', 'SPb', '2023-01-01 09:00:00'),
    ('M', 'Kazan', '2023-01-01 10:00:00'),
    ('F', 'Moscow', '2023-01-01 11:00:00'),
    ('M', 'SPb', '2023-01-01 12:00:00');

-- Вставка данных в таблицу employee_data_rmt
INSERT INTO employee_data_rmt (Gender, City, Start_Date) VALUES 
    ('M', 'Moscow', '2023-01-01 08:00:00'),
    ('F', 'SPb', '2023-01-01 09:00:00'),
    ('M', 'Kazan', '2023-01-01 10:00:00'),
    ('F', 'Moscow', '2023-01-01 11:00:00'),
    ('M', 'SPb', '2023-01-01 12:00:00');


-- Добавим дубликаты в таблицу employee_data_mt для тестирования MergeTree
INSERT INTO employee_data_mt (Gender, City, Start_Date) VALUES 
    ('F', 'Moscow', '2023-01-01 11:00:00'),  
    ('M', 'SPb', '2023-01-01 12:00:00'),   
    ('M', 'Moscow', '2023-01-01 08:00:00');

-- Добавим дубликаты в таблицу employee_data_rmt для тестирования ReplacingMergeTree
INSERT INTO employee_data_rmt (Gender, City, Start_Date) VALUES 
    ('F', 'Moscow', '2023-01-01 11:00:00'),  
    ('M', 'SPb', '2023-01-01 12:00:00'),   
    ('M', 'Moscow', '2023-01-01 08:00:00'); 

OPTIMIZE TABLE employee_data_rmt FINAL;

SELECT 
  (SELECT COUNT() FROM employee_data_mt) as cnt_mt,
  (SELECT COUNT() FROM employee_data_rmt) as cnt_rmt;

```

<b>Вывод:</b>

1.MergeTree хранит все вставленные данные, включая дубликаты.

2.ReplacingMergeTree хранит только одну запись на Start_Date после слияния (FINAL).

3.Использовать MergeTree стоит, если важно сохранять все версии данных. ReplacingMergeTree подходит, когда необходимо автоматически убирать дубликаты и оставлять только одну актуальную запись.